In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Updated Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

# Load Pre-Trained Embeddings Manually

In [4]:
from torchtext.vocab import Vectors, Vocab

text = "I love this film very much ."
with open('vector_cache/test-vecs.txt', 'w', encoding='utf-8') as f:
    for token in text.split():
        vec = ["%.6f" % vi for vi in np.random.randn(5)]
        f.write(" ".join([token] + vec))
        f.write("\n")

vecs = Vectors('test-vecs.txt', cache='vector_cache')
print(vecs.stoi)
print(vecs.itos)
print(vecs.vectors)

{'I': 0, 'love': 1, 'this': 2, 'film': 3, 'very': 4, 'much': 5, '.': 6}
['I', 'love', 'this', 'film', 'very', 'much', '.']
tensor([[ 1.3452,  0.3613, -0.1679,  0.5251, -0.2278],
        [ 0.2983,  0.5024, -0.4676,  0.1557,  1.2104],
        [-0.4675,  1.4933,  0.4371, -0.5939,  0.1841],
        [ 0.1193,  0.3751,  0.4939,  1.8917,  2.2347],
        [ 0.4038, -0.6436,  0.0377, -0.5992, -0.3922],
        [-0.0037,  1.7008, -0.3855, -0.4128, -0.9682],
        [ 0.1683,  1.4306,  1.2625,  1.1546,  1.0423]])


In [5]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

# Set `include_lengths=True` to return the text lengths too. 
TEXT = Field(tokenize='spacy', include_lengths=True)
GENDER = Field()
LABEL = LabelField(dtype=torch.float)

# Create a Dataset
fields = [('text', TEXT), ('gender', GENDER), ('label', LABEL)]
raw_data = [["I like this film.", "f", "pos"], 
            ["I hate it.", "f", "neg"], 
            ["I have no feelings about it.", "m", "neg"], 
            ["It is my best.", "m", "pos"], 
            ["My father loves it so much and I do think so.", "f", "pos"]]

examples = [Example.fromlist(d, fields) for d in raw_data]
data = Dataset(examples, fields)
data

In [6]:
TEXT.build_vocab(data)
GENDER.build_vocab(data)
LABEL.build_vocab(data)

print(TEXT.vocab.vectors)

None


In [7]:
# The missing tokens are initialized as zeros, or by `unk_init` if provided. 
TEXT.build_vocab(data, vectors=vecs)
print(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1683,  1.4306,  1.2625,  1.1546,  1.0423],
        [ 1.3452,  0.3613, -0.1679,  0.5251, -0.2278],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1193,  0.3751,  0.4939,  1.8917,  2.2347],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 

In [8]:
BATCH_SIZE = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
iterator = BucketIterator(data, batch_size=BATCH_SIZE, device=device, shuffle=True)

# `batch.text` is now a tuple with the second element being the text lengths.
for batch in iterator:
    print(batch.text)

(tensor([[ 7],
        [12],
        [19],
        [ 4],
        [ 5],
        [20],
        [ 9],
        [ 3],
        [11],
        [23],
        [ 5],
        [ 2]]), tensor([12]))
(tensor([[ 3,  3],
        [18, 15],
        [24,  4],
        [14,  2],
        [ 2,  1]]), tensor([5, 4]))
(tensor([[ 3,  6],
        [16, 17],
        [22, 21],
        [13, 10],
        [ 8,  2],
        [ 4,  1],
        [ 2,  1]]), tensor([7, 5]))


# Use Pre-Trained Embeddings

In [9]:
import torchtext

# Set `include_lengths=True` to return the text-lengths too. 
TEXT = Field(tokenize='spacy', include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [10]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [11]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# NOTE: `sort_within_batch=True` for packed padded sequences
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, sort_within_batch=True, device=device)

# Build Model
## Different RNN Architecture: LSTM
## Bidirectional RNN
## Multi-Layer RNN
## Regularization: Dropout

## Implementation Details
### How to ignore the `<pad>` token?
* Pass the index of `<pad>` token as `padding_idx` argument to `nn.Embedding`
* Pass packed padded sequences to RNNs
* Attention?
* `ignore_index` of loss function?

### The `hidden` output of LSTM
The `hidden` has a shape of **(num_layers * num_directions, batch_size, hid_dim)**.  
These are ordered in **(forward_layer_0, backward_layer_0, forward_layer_1, backward_layer_1, ..., forward_layer_n, backward_layer_n)**, from bottom to top.  

In [12]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, out_dim, n_layers, bidirect, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, 
                           bidirectional=bidirect, dropout=dropout)
        # `bidirectional=True` by default
        self.fc = nn.Linear(hid_dim*2, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.dropout(self.emb(text))
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lens)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        packed_outs, (hidden, cell) = self.rnn(packed_embedded)
        # Unpack sequence, NOT used here. 
        outs, out_lens = nn.utils.rnn.pad_packed_sequence(packed_outs)

        hidden = self.dropout(torch.cat([hidden[-2], hidden[-1]], dim=-1))
        return self.fc(hidden)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
HID_DIM = 256
OUT_DIM = 1
N_LAYERS = 2
BIDIRECT = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifer = Classifier(IN_DIM, EMB_DIM, HID_DIM, OUT_DIM, 
                       N_LAYERS, BIDIRECT, DROPOUT, PAD_IDX).to(device)
count_parameters(classifer)

4810857

### Initialize Embeddings with Pre-Trained Vectors
* Replace the embedding layer's weight with `TEXT.vocab.vectors`.  
    * **Note:** Use `weight.data` for data Initialization.  
* Explicitly Initialize the embeddings of `<unk>` and `<pad>` tokens to zeros.  
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [14]:
print(TEXT.vocab.vectors.size())
print(classifer.emb.weight.size())

print(TEXT.vocab.vectors[:5, :8])
print(classifer.emb.weight[:5, :8])
classifer.emb.weight.data.copy_(TEXT.vocab.vectors)
print(classifer.emb.weight[:5, :8])

torch.Size([25002, 100])
torch.Size([25002, 100])
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878],
        [-0.4843,  1.6366, -1.4119, -0.4014, -1.1278,  1.6279,  0.7653, -0.5195],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]])
tensor([[ 0.3339, -0.5884,  1.3674, -0.3534,  0.9094,  1.0981,  1.2600, -0.5185],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.6838, -0.8789, -0.0712,  0.7410, -0.5568,  1.3102,  0.9170, -0.3381],
        [ 0.5380, -1.2371, -0.0317,  0.0616,  0.5156, -0.4544,  0.0418, -0.2760],
        [-1.6648,  0.4871,  1.2823, -0.6593, -0.5190, -0.9394,  0.8888,  0.2326]],
       grad_fn=<SliceBackward>)
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878],
        [-0.48

In [15]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifer.emb.weight.data[UNK_IDX].zero_()
classifer.emb.weight.data[PAD_IDX].zero_()

print(classifer.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       grad_fn=<SliceBackward>)


# Train Model

In [16]:
optimizer = optim.Adam(classifer.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [17]:
def train_epoch(classifer, iterator, optimizer, loss_func):
    classifer.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifer(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifer, iterator, loss_func):
    classifer.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifer(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

SyntaxError: invalid syntax (<ipython-input-17-df5768e69eba>, line 21)

In [18]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = epoch_secs // 60, int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')